# Как взвесить осла?

Ослы играют важную роль в сельской местности Кении. Они нужны людям для перевозки урожая и воды, вспашки полей и для личного транспорта. Когда осел заболевает, ветеринару необходимо выяснить, сколько весит осел, чтобы назначить нужное количество лекарства.

Но у многих ветеринаров в сельской местности Кении нет доступа к весам, чтобы взвесить осла, поэтому им нужно угадать, сколько весит осел.

Слишком мало лекарств и инфекция может повториться, слишком много лекарств - можно вызвать опасную передозировку.

В Кении насчитывается более 1,8 миллиона ослов, поэтому важно иметь простой и точный способ оценки веса осла.

Поля датасета:
1. BCS - оценка состояния тела: от 1 (истощенный) через 3 (нормальный) до 5 (страдающий ожирением) с шагом 0.5;
2. Age - возраст в годах, младше 2, 2-5, 5-10, 10-15, 15-20, и старше 20 лет;
4. Sex - пол;
5. Length - длинна тела (см);
6. Girth - обхват (по шее в см);
7. Height - высота в холке (см);
8. Weight - вес осла (целевая переменная).

Обхват измеряется вокруг туловища сразу за передними ногами, высота измеряется от земли до того места, где шея соединяется с верхней частью спины, а длина измеряется от переднего локтевого сустава до задней части таза.

In [ ]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке,
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
#
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию,
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными,
# которые необходимо произвести, что бы сопоставить признаки целевому значению.


# Область работы 1 (библиотеки)

In [ ]:
# Данный блок в области 1 НЕ выполняется преподавателем
#
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import pandas as pd
from IPython.display import display, Image
import seaborn as sns
from scipy.stats import gaussian_kde

from sklearn.compose import ColumnTransformer , make_column_transformer, TransformedTargetRegressor
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.model_selection import cross_val_score, KFold, StratifiedShuffleSplit, GridSearchCV
from sklearn.pipeline import Pipeline,make_pipeline

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer,PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,QuantileTransformer,PowerTransformer
from sklearn.datasets import make_classification
from sklearn.metrics import precision_score, classification_report, confusion_matrix, ConfusionMatrixDisplay,log_loss
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,mean_absolute_percentage_error,f1_score,accuracy_score
from sklearn.metrics import recall_score, zero_one_loss, roc_auc_score, balanced_accuracy_score
import category_encoders as ce
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score, mean_absolute_percentage_error

from category_encoders import TargetEncoder
from sklearn.linear_model import ElasticNet, Ridge, Lasso, Lars,SGDRegressor,LogisticRegression, LinearRegression
from sklearn.svm import SVR,SVC

from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingRegressor,VotingRegressor
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier

from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer , make_column_transformer, TransformedTargetRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn import svm
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler,LabelEncoder,FunctionTransformer

from sklearn.neighbors import LocalOutlierFactor
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest, f_classif

import pickle

import time
import re as re

from matplotlib.colors import ListedColormap
from collections import OrderedDict

# pip install category-encoders

from sklearn import tree

import sklearn
print(sklearn.__version__)

1.2.2


# Область работы 2 (поиск модели .... )

In [5]:
# Данный блок(и) НЕ выполняются преподавателем в области 2
# блок(и) предназначены для поиска лучшей модели
# должен быть понятен и очевиден отбор параметров модели
# оставляйте свои комментарии и разъяснения
#
# Запрещается размещать данные блоки за пределами обасти 2
# Все блоки данной области должны быть выполнены
#
# ЗАПРЕЩАЕТСЯ ИСПОЛЬЗОВАТЬ ТЕСТОВЫЙ НАБОР
#
# Путь к тренировочному набору
#
# содержит только имя файла, без имен папок !!!

df = pd.read_csv("train.csv")
df.head()

,BCS,Age,Sex,Length,Girth,Height,Weight
0,2.5,10-15,gelding,89,121,106,158
1,2.5,10-15,stallion,98,117,105,145
2,2.5,2-5,stallion,89,107,100,109
3,2.5,10-15,female,98,120,103,170
4,2.5,2-5,female,87,101,93,110


from matplotlib import pyplot as plt
_df_0['Length'].plot(kind='hist', bins=20, title='Length')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Girth'].plot(kind='hist', bins=20, title='Girth')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['Height'].plot(kind='hist', bins=20, title='Height')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['Weight'].plot(kind='hist', bins=20, title='Weight')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Sex'].value_counts()
    for x_label, grp in _df_17.groupby('Age')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Age')
_ = plt.ylabel('Sex')

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['Age'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='Length', y='Age', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['Sex'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='Length', y='Sex', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['Age'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='Girth', y='Age', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_21['Sex'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_21, x='Girth', y='Sex', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [15]:
display(df['Sex'].unique())
df['Sex'].nunique()

array(['gelding', 'stallion', 'female'], dtype=object)

3

In [6]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   BCS     380 non-null    float64
 1   Age     380 non-null    object 
 2   Sex     380 non-null    object 
 3   Length  380 non-null    int64  
 4   Girth   380 non-null    int64  
 5   Height  380 non-null    int64  
 6   Weight  380 non-null    int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 20.9+ KB


None

In [7]:
missing_values = df.isnull().sum()
print("Количество пропущенных значений по признакам:")
print(missing_values)

Количество пропущенных значений по признакам:
BCS       0
Age       0
Sex       0
Length    0
Girth     0
Height    0
Weight    0
dtype: int64


In [9]:
print (df[["Age", "Weight"]].groupby(['Age'], as_index=False).mean())

     Age      Weight
0  10-15  164.213018
1  15-20  168.137931
2    2-5  137.183099
3   5-10  154.897959
4     <2   99.848485
5    >20  158.413793


In [11]:
df.drop_duplicates(inplace = True)
df.duplicated().sum()
df.head()

,BCS,Age,Sex,Length,Girth,Height,Weight
0,2.5,10-15,gelding,89,121,106,158
1,2.5,10-15,stallion,98,117,105,145
2,2.5,2-5,stallion,89,107,100,109
3,2.5,10-15,female,98,120,103,170
4,2.5,2-5,female,87,101,93,110


In [12]:
cat_features = ['Sex', 'Age']
num_features = ['BCS', 'Height', 'Girth', 'Length']

In [13]:
X = df.drop(columns = ['Weight'], axis=1)
y = df['Weight']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 52)

In [17]:
transformer_cat = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse_output = False, handle_unknown='ignore').set_output(transform='pandas')),
    ('scaler', StandardScaler())
])

transformer_num = Pipeline(steps=[
    ('scaler', StandardScaler())
])

In [18]:
CT = ColumnTransformer([
        ("num", transformer_num, num_features),
        ("cat", transformer_cat, cat_features),
        ])

display(CT)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['BCS', 'Height', 'Girth', 'Length']),
                                ('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['Sex', 'Age'])])

In [19]:
CT.set_output(transform ='pandas')
res_ct = CT.fit_transform(X_train)
pd.DataFrame(res_ct).head()

,num__BCS,num__Height,num__Girth,num__Length,cat__Sex_female,cat__Sex_gelding,cat__Sex_stallion,cat__Age_10-15,cat__Age_15-20,cat__Age_2-5,cat__Age_5-10,cat__Age_<2,cat__Age_>20
314,1.460404,0.170518,0.790527,0.770185,1.119927,-0.439119,-0.807573,1.119927,-0.285133,-0.504695,-0.363201,-0.307653,-0.285133
25,-0.959954,0.170518,-0.752814,-0.920799,1.119927,-0.439119,-0.807573,-0.892915,-0.285133,-0.504695,2.753297,-0.307653,-0.285133
31,-0.959954,0.170518,-1.267260,-0.400496,1.119927,-0.439119,-0.807573,-0.892915,-0.285133,1.981395,-0.363201,-0.307653,-0.285133
26,0.250225,0.617392,0.919139,1.550639,-0.892915,-0.439119,1.238278,-0.892915,3.507136,-0.504695,-0.363201,-0.307653,-0.285133
104,0.250225,-0.499794,-0.366979,0.249882,1.119927,-0.439119,-0.807573,-0.892915,-0.285133,1.981395,-0.363201,-0.307653,-0.285133


In [22]:
regressors = [KNeighborsRegressor(), RandomForestRegressor(), LinearRegression(), GradientBoostingRegressor(), SVR()]

pipe = Pipeline([('preprocessing', CT),
                 ('regressors', KNeighborsRegressor())])

param_grid = [
    {
        'regressors': regressors
    },
]

grid = GridSearchCV(pipe, param_grid, cv = KFold(5), return_train_score = True, scoring = 'neg_root_mean_squared_error', n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         ['BCS',
                                                                          'Height',
                                                                          'Girth',
                                                                          'Length']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False)),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['Sex',
                                                                          'Age'])])),
                                       ('regressors', KNeighborsRegressor())]),
             n_jobs=-1,
             param_grid=[{'regressors': [KNeighborsRegressor(),
                                         RandomForestRegressor(),
                                         LinearRegression(),
                                         GradientBoostingRegressor(), SVR()]}],
             return_train_score=True, scoring='neg_root_mean_squared_error')

In [23]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

,2,3,1,0,4
mean_fit_time,0.029259,0.198332,0.59993,0.051893,0.092815
std_fit_time,0.010859,0.038997,0.085282,0.012534,0.049704
mean_score_time,0.014513,0.029419,0.064421,0.032554,0.046457
std_score_time,0.003546,0.007716,0.01014,0.00829,0.020671
param_regressors,LinearRegression(),GradientBoostingRegressor(),RandomForestRegressor(),KNeighborsRegressor(),SVR()
params,{'regressors': LinearRegression()},{'regressors': GradientBoostingRegressor()},{'regressors': RandomForestRegressor()},{'regressors': KNeighborsRegressor()},{'regressors': SVR()}
split0_test_score,-9.579257,-10.745469,-11.59907,-14.724734,-23.162416
split1_test_score,-8.275224,-8.903819,-10.811279,-11.137054,-19.377242
split2_test_score,-8.83347,-8.23207,-8.406919,-13.173931,-19.165078
split3_test_score,-9.614601,-10.225961,-10.59917,-15.874722,-28.236122


In [42]:
gb = Pipeline([('preprocessing', CT), ('regressor', GradientBoostingRegressor(
                                    learning_rate=0.01,
                                subsample=0.1,
                                min_samples_split = 2,
                                max_depth = 3,
                                  n_estimators=1000,
                                random_state=52))])


gb.fit(X_train, y_train)
y_pred_best = gb.predict(X_test)
y_pred_train = gb.predict(X_train)



print('Правильность на тренировочном наборе:', mean_absolute_percentage_error(y_train, y_pred_train))
print('Правильность на тестовом наборе:', mean_absolute_percentage_error(y_test, y_pred_best))

Правильность на тренировочном наборе: 0.036766420653024386
Правильность на тестовом наборе: 0.04582730194965625


In [43]:
lr = Pipeline([('preprocessing', CT), ('regressor', LinearRegression(
                                    fit_intercept = True,
                                    n_jobs = -1
                                    ))])


lr.fit(X_train, y_train)
y_pred_best = gb.predict(X_test)
y_pred_train = gb.predict(X_train)



print('Правильность на тренировочном наборе:', mean_absolute_percentage_error(y_train, y_pred_train))
print('Правильность на тестовом наборе:', mean_absolute_percentage_error(y_test, y_pred_best))

Правильность на тренировочном наборе: 0.036766420653024386
Правильность на тестовом наборе: 0.04582730194965625


In [50]:
rf = Pipeline([('preprocessing', CT), ('regressor', RandomForestRegressor(n_estimators = 1000,
                                                                            max_depth = 3,
                                                                             max_features = 8,
                                                                            min_samples_split = 2,
                                                                            random_state = 42,
                                                                            n_jobs = -1))])

rf.fit(X_train, y_train)
y_pred_best = rf.predict(X_test)
y_pred_train = rf.predict(X_train)



print('Правильность на тренировочном наборе:', mean_absolute_percentage_error(y_train, y_pred_train))
print('Правильность на тестовом наборе:', mean_absolute_percentage_error(y_test, y_pred_best))

Правильность на тренировочном наборе: 0.0519827133293076
Правильность на тестовом наборе: 0.057639641543853724


In [51]:
transformer_cat = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse_output = False, handle_unknown='ignore').set_output(transform='pandas')),
    ('scaler', StandardScaler())
])

transformer_num = Pipeline(steps=[
    ('scaler', StandardScaler())
])

CT = ColumnTransformer([
        ("num", transformer_num, num_features),
        ("cat", transformer_cat, cat_features),
        ])

display(CT)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['BCS', 'Height', 'Girth', 'Length']),
                                ('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['Sex', 'Age'])])

In [52]:
start = time.time()

scalers = [MinMaxScaler(), StandardScaler(), RobustScaler()]

pipe = Pipeline([('preprocessing', CT), ('regressor', GradientBoostingRegressor(
                                    learning_rate=0.01,
                                subsample=0.9,
                                min_samples_split = 2,
                                max_depth = 3,
                                  n_estimators=800,
                                random_state=52))])


cross_val = KFold(5)



param_grid =[{
    'preprocessing__num__scaler' : scalers,
    'preprocessing__cat__scaler' : scalers,
    }]

grid = GridSearchCV(pipe, param_grid, n_jobs = -1, cv = cross_val,
                    return_train_score = True, scoring = 'neg_root_mean_squared_error')

grid.fit(X_train, y_train)

print(f"цикл пройден за {round((time.time() - start)/60, 1)} минут")

цикл пройден за 0.5 минут


In [53]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

,1,4,7,2,5,8,0,3,6
mean_fit_time,1.006088,0.84622,0.869544,0.829702,1.269621,0.798632,1.523613,1.056224,0.844882
std_fit_time,0.164776,0.024767,0.01092,0.009672,0.119032,0.057864,0.294755,0.27153,0.013863
mean_score_time,0.016855,0.014204,0.01151,0.011537,0.016111,0.010676,0.020017,0.01245,0.011524
std_score_time,0.009734,0.005872,0.000339,0.000411,0.004652,0.001163,0.007765,0.002494,0.00016
param_preprocessing__cat__scaler,MinMaxScaler(),StandardScaler(),RobustScaler(),MinMaxScaler(),StandardScaler(),RobustScaler(),MinMaxScaler(),StandardScaler(),RobustScaler()
param_preprocessing__num__scaler,StandardScaler(),StandardScaler(),StandardScaler(),RobustScaler(),RobustScaler(),RobustScaler(),MinMaxScaler(),MinMaxScaler(),MinMaxScaler()
params,"{'preprocessing__cat__scaler': MinMaxScaler(),...",{'preprocessing__cat__scaler': StandardScaler(...,"{'preprocessing__cat__scaler': RobustScaler(),...","{'preprocessing__cat__scaler': MinMaxScaler(),...",{'preprocessing__cat__scaler': StandardScaler(...,"{'preprocessing__cat__scaler': RobustScaler(),...","{'preprocessing__cat__scaler': MinMaxScaler(),...",{'preprocessing__cat__scaler': StandardScaler(...,"{'preprocessing__cat__scaler': RobustScaler(),..."
split0_test_score,-10.519174,-10.519174,-10.519174,-10.586114,-10.586114,-10.586114,-10.565211,-10.565211,-10.565211
split1_test_score,-8.81127,-8.81127,-8.81127,-8.812697,-8.812697,-8.812697,-8.805454,-8.805454,-8.805454
split2_test_score,-8.138526,-8.138526,-8.138526,-8.177076,-8.177076,-8.177076,-8.130719,-8.130719,-8.130719


In [54]:
transformer_cat = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse_output = False, handle_unknown='ignore').set_output(transform='pandas')),
    ('scaler', MinMaxScaler())
])

transformer_num = Pipeline(steps=[
    ('scaler', StandardScaler())
])

CT = ColumnTransformer([
        ("num", transformer_num, num_features),
        ("cat", transformer_cat, cat_features),
        ])

display(CT)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['BCS', 'Height', 'Girth', 'Length']),
                                ('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False)),
                                                 ('scaler', MinMaxScaler())]),
                                 ['Sex', 'Age'])])

In [56]:
gb = Pipeline([('preprocessing', CT), ('regressor', GradientBoostingRegressor(
                                    learning_rate=0.01,
                                subsample=0.1,
                                min_samples_split = 2,
                                max_depth = 3,
                                  n_estimators=1000,
                                random_state=52))])


gb.fit(X_train, y_train)
y_pred_best = gb.predict(X_test)
y_pred_train = gb.predict(X_train)



print('Правильность на тренировочном наборе:', mean_absolute_percentage_error(y_train, y_pred_train))
print('Правильность на тестовом наборе:', mean_absolute_percentage_error(y_test, y_pred_best))

Правильность на тренировочном наборе: 0.036766420653024386
Правильность на тестовом наборе: 0.04582730194965625
